In [81]:
%pip install llama-index-embeddings-azure-openai llama-index-llms-azure-openai llama-index-vector-stores-chroma llama-index chromadb


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# 1. Immer ausführen

In [82]:
from llama_index.core import StorageContext, VectorStoreIndex, SimpleDirectoryReader, load_index_from_storage
from llama_index.core.node_parser import  SentenceSplitter
from llama_index.readers.file.flat.base import FlatReader
import chromadb
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate
from llama_index.core.retrievers import VectorIndexRetriever, AutoMergingRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
import os
from dotenv import load_dotenv
from custom_transformer import extract_metadata, extract_pages


In [83]:
load_dotenv()

True

In [84]:
fpath = 'Rechnungshofberichte'
chromapath = 'Vektordatenbank'
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("OPENAI_API_KEY")
api_version = os.getenv("OPENAI_API_VERSION")
embedding_deployment_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT") or "text-embedding-3-large"

In [85]:
embedding_model = AzureOpenAIEmbedding(
    model="text-embedding-3-large",
    deployment_name=embedding_deployment_name,
    azure_endpoint = azure_endpoint,
    api_key = api_key,
    api_version = api_version,
)

llm = AzureOpenAI(
    model="gpt-4o-mini",
    deployment_name = "gpt-4o-mini",
    azure_endpoint = azure_endpoint,
    api_key = api_key,
    api_version=api_version
)

# 2. Nur wenn der Index neuaugebaut werden muss ansonsten gehe zu 3.

In [86]:
#Dokumente laden
dir_reader = SimpleDirectoryReader(
    input_dir=fpath,
    file_extractor = {'.md': FlatReader()},
    recursive=True
)
documents=[]
for doc in dir_reader.load_data():
    metadata = extract_metadata(doc.metadata.get("file_path",""))
    doc.metadata.update(metadata)
    
    doc_splitted = extract_pages([doc])
    documents.extend(doc_splitted)
    

#Chunks erstellen
parsing_method = 'SentenceSplitter'
if parsing_method == 'SentenceSplitter':
    node_parser = SentenceSplitter(
        chunk_size=1024,
        chunk_overlap = 30,
    )
nodes = node_parser.get_nodes_from_documents(documents)

#ChromaDB initialisieren
chroma_client = chromadb.PersistentClient(path=chromapath)
chroma_collection = chroma_client.get_or_create_collection("Jahresberichte")
print(f"START: Chroma Collection hat {chroma_collection.count()} Vektoren.")

vector_store = ChromaVectorStore(
    chroma_collection=chroma_collection,
    mode="append"
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

#Zusätzliches Preprocessing durchführen (Metadaten, Zusammenfassung, ..)
transformations = [
    node_parser,
]

#Index erstellen
index = VectorStoreIndex.from_documents(
    documents=documents,
    transformations=transformations,
    storage_context=storage_context,
    embed_model=embedding_model,
    show_progress=True,
)
index.storage_context.persist(persist_dir=chromapath)
print(f"ENDE: Chroma hat nun {chroma_collection.count()} Vektoren.")


START: Chroma Collection hat 48830 Vektoren.





































































































































































Parsing nodes: 100%|██████████| 34795/34795 [00:20<00:00, 1668.56it/s]



NotFoundError: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}

# 3. Laden des gespeicherten Index

In [ ]:
chroma_client = chromadb.PersistentClient(path=chromapath)
chroma_collection = chroma_client.get_or_create_collection("Jahresberichte")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store, persist_dir=chromapath)
index = load_index_from_storage(storage_context=storage_context, embed_model=embedding_model)

Loading llama_index.core.storage.kvstore.simple_kvstore from Vektordatenbank/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from Vektordatenbank/index_store.json.


# 4. Ab hier immer ausführen

In [ ]:
qa_prompt_str = (
    "Das ist die Datengrundlage:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Beantworte die folgende Frage nur anhand der Datengrundlage: "
    "{query_str}\n"
)

refine_prompt_str = (
    "Falls möglich, verbessere die Antwort anhand dieser Datengrundlage:\n"
    "------------\n"
    "{query_str}\n "
    "Falls die Antwort nicht hilfreich ist, verwende wieder die ursprüngliche Antwort.\n"
    "ursprüngliche Antwort: {existing_answer}"
)

#Chat templates definieren
chat_text_qa_msgs = [
    ChatMessage(role=MessageRole.SYSTEM, content="Du bist ein KI-Assistent der Berliner Verwaltung, der auf Basis einer Datengrundlage sinnvolle Antworten generiert.\n"
                                                 "Beachte die gegebene Datengrundlage, fokussiere dich auf relevante Inhalte und verändere NIEMALS Fakten, Namen, Berufsbezeichnungen, Zahlen oder Datumsangaben."),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

chat_refine_msgs = [
    ChatMessage(role=MessageRole.SYSTEM, content="Du bist ein KI-Assistent der Berliner Verwaltung, der auf Basis einer Datengrundlage sinnvolle Antworten generiert.\n"
                                                 "Beachte die gegebene Datengrundlage, fokussiere dich auf relevante Inhalte und verändere NIEMALS Fakten, Namen, Berufsbezeichnungen, Zahlen oder Datumsangaben."),
    ChatMessage(role=MessageRole.USER, content=refine_prompt_str),
]

refine_template = ChatPromptTemplate(chat_refine_msgs)


In [ ]:
def custom_query_engine(
        index,
        text_qa_template=None,
        llm=None,
):

    response_synthesizer = get_response_synthesizer(
        text_qa_template=text_qa_template,
        llm=llm,
        response_mode="compact"
    )

    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k= 5,
    )

    query_engine = RetrieverQueryEngine(
        retriever=retriever,
        response_synthesizer=response_synthesizer,
    )

    return query_engine

In [ ]:
user_input = 'Welche Coronasoforthilfen wurden von den verschiedenen Rechnungshöfen geprüft? '
response = custom_query_engine(index, text_qa_template, llm).query(user_input)
print(response.response)

Der Rechnungshof hat in den Jahren 2022/2023 die Bearbeitung der Steuerfälle mit erhaltenen Corona-Hilfen in den Finanzämtern Prenzlauer Berg und für Körperschaften IV geprüft. Dabei lag ein besonderer Fokus auf den Corona-Soforthilfen II. In 78 von 289 Steuerfällen im Finanzamt Prenzlauer Berg und in 75 von 263 Steuerfällen im Finanzamt für Körperschaften IV wurde festgestellt, dass die Anspruchsvoraussetzungen für die erhaltenen Corona-Soforthilfen II voraussichtlich nicht erfüllt waren.


In [ ]:
from metrics import calculate_recall_and_positions

references = [
  {"ort": "Baden-Württemberg", "jahr": "2022", "page": 117},
  {"ort": "Baden-Württemberg", "jahr": "2022", "page": 118}
]

resp = [{"ort":node.node.metadata['ort'], "jahr": node.node.metadata['jahr'], "page": node.node.metadata['seite']} for node in response.source_nodes]
recall, position = calculate_recall_and_positions( reference_items=references, predicted_items=resp)

print(f"recall: {recall}, relevant_positions: {position}")

recall: 0.0, relevant_positions: []


In [ ]:
for node in response.source_nodes:
    print(f"🧾 Metadaten: Typ: {node.node.metadata['typ']}, Ort: {node.node.metadata['ort']}, Jahr: {node.node.metadata['jahr']}, Seite: {node.node.metadata['seite']}")
    print("🔢 Score:", node.score)
    print("-" * 40)

🧾 Metadaten: Typ: Jahresberichte, Ort: Bremen, Jahr: 2022, Seite: 61
🔢 Score: 0.5312121804845938
----------------------------------------
🧾 Metadaten: Typ: Jahresberichte, Ort: Berlin, Jahr: 2021, Seite: 48
🔢 Score: 0.5138158245977779
----------------------------------------
🧾 Metadaten: Typ: Jahresberichte, Ort: Berlin, Jahr: 2021, Seite: 48
🔢 Score: 0.5125222595351139
----------------------------------------
🧾 Metadaten: Typ: Jahresberichte, Ort: Baden-Württemberg, Jahr: 2020, Seite: 132
🔢 Score: 0.5115802709600407
----------------------------------------
